In [23]:
# #############################
# Draft_3
# Author  : Bosen Li
# Date    : Feb 11, 2025
# Notes   : n/a
# #############################

In [ ]:
import pandas as pd
import datetime as dt
import wrds
# conn = wrds.Connection()
#Y@QC8f4qEjCz!eY 

In [25]:
# downloading a identifier conversion table so we can splice in gvkeys

query = """ SELECT * FROM ciq.wrds_gvkey """
gvkey = conn.raw_sql(query)

# the following two blocks fill in null dates with either very early or very late dates
# this is needed for the upcoming merge

a = gvkey[pd.isnull(gvkey.startdate)].copy()
gvkey = gvkey[~pd.isnull(gvkey.startdate)]
a.startdate = dt.date(1901,1,1)
gvkey = pd.concat([a, gvkey])

a = gvkey[pd.isnull(gvkey.enddate)].copy()
gvkey = gvkey[~pd.isnull(gvkey.enddate)]
a.enddate = dt.date(2999,12,31)
gvkey = pd.concat([a, gvkey])

print(f'\nThe number of rows is {len(gvkey):,}.\n')
gvkey.tail(10)


The number of rows is 135,556.



,companyid,gvkey,startdate,enddate,companyname
112831,325748154.0,162285,2016-02-09,2021-08-02,"ABVC BioPharma, Inc."
113132,330819854.0,194560,2016-03-10,2016-10-17,"Glorywin Entertainment Group, Inc."
113227,331993896.0,241714,2016-09-01,2019-03-31,"Daiyu LIC Holdings Co., Ltd."
113499,364606463.0,003785,2017-05-24,2017-06-25,U.S. Gold Corp.
113691,372095322.0,107953,2017-01-01,2017-01-15,Falcon Energy Materials plc
115488,427752809.0,026296,2017-06-10,2017-06-13,"LXRandCo, Inc."
117399,549194384.0,235136,2022-06-04,2024-04-25,Kinarus Therapeutics Holding AG
123451,667517908.0,034597,2020-05-23,2023-07-06,Luduson G Inc.
125101,696209709.0,035079,2022-11-15,2024-10-15,AGBA Group Holding Limited
126129,705855838.0,187757,2020-11-07,2023-09-11,"The Graystone Company, Inc."


In [26]:
# This block pulls down all the S&P ratings. 
# Note that we are pulling down the alpha numeric versions

query = """
SELECT company_id as companyid, entity_pname, ratingdate, ratingsymbol, ratingactionword, unsol
FROM ciq_ratings.wrds_erating
WHERE longtermflag = 1 AND ratingtypename = 'Local Currency LT' AND ratingdate >= '1990-01-01'
"""

ratings = conn.raw_sql(query)

symbols = ['AAA', 'AA+', 'AA', 'AA-', 'A+', 'A', 'A-', 'BBB+', 'BBB', 'BBB-', 
           'BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'CCC+', 'CCC', 'CCC-', 'CC', 
           'C', 'D', 'SD', 'NR', 'R']
ratings = ratings[ratings.ratingsymbol.isin(symbols)]
print(f'\nThe number of rows is {len(ratings):,}.\n')
ratings.head(10)


The number of rows is 239,212.



,companyid,entity_pname,ratingdate,ratingsymbol,ratingactionword,unsol
0,168569.0,American Airlines Group Inc.,1990-08-20,A-,Downgrade,N
1,168569.0,American Airlines Group Inc.,1990-08-20,A-,<NA>,N
2,168569.0,American Airlines Group Inc.,1990-08-20,A-,<NA>,N
3,168569.0,American Airlines Group Inc.,1990-08-20,A-,<NA>,N
4,168569.0,American Airlines Group Inc.,1991-04-05,BBB+,Downgrade,N
5,168569.0,American Airlines Group Inc.,1991-04-05,BBB+,<NA>,N
6,168569.0,American Airlines Group Inc.,1992-06-01,BBB,Downgrade,N
7,168569.0,American Airlines Group Inc.,1993-03-11,BB+,<NA>,N
8,168569.0,American Airlines Group Inc.,1993-03-11,BB+,<NA>,N
9,168569.0,American Airlines Group Inc.,1993-03-11,BB+,<NA>,N


In [27]:
### merge ratings and gvkey

ratings2 = pd.merge(gvkey[['gvkey', 'companyid', 'startdate', 'enddate']], ratings, on = 'companyid')
print(f'\nThe number of rows is {len(ratings2):,}.\n')
ratings2


The number of rows is 90,586.



,gvkey,companyid,startdate,enddate,entity_pname,ratingdate,ratingsymbol,ratingactionword,unsol
0,210835,18511.0,1901-01-01,2999-12-31,3i Group PLC,1995-12-07,AA-,New Rating,N
1,210835,18511.0,1901-01-01,2999-12-31,3i Group PLC,1995-12-07,AA-,<NA>,N
2,210835,18511.0,1901-01-01,2999-12-31,3i Group PLC,2001-12-20,A+,Downgrade,N
3,210835,18511.0,1901-01-01,2999-12-31,3i Group PLC,2001-12-20,A+,<NA>,N
4,210835,18511.0,1901-01-01,2999-12-31,3i Group PLC,2001-12-20,A+,<NA>,N
...,...,...,...,...,...,...,...,...,...
90581,019045,41269602.0,1994-10-02,2005-09-30,Mitsubishi Chemical Corp.,2007-04-20,BBB,Upgrade,N
90582,019045,41269602.0,1994-10-02,2005-09-30,Mitsubishi Chemical Corp.,2008-07-15,NR,Not Rated,Y
90583,019045,41269602.0,1994-10-02,2005-09-30,Mitsubishi Rayon Co. Ltd.,2004-03-19,BBB,New Rating,N
90584,019045,41269602.0,1994-10-02,2005-09-30,Mitsubishi Rayon Co. Ltd.,2006-06-07,BBB+,Upgrade,N


In [48]:
## drop duplicates
ratings3 = ratings2.drop_duplicates(subset=['gvkey', 'ratingdate'])
print(f'\nThe number of rows is {len(ratings3):,}.\n')
ratings3


The number of rows is 46,208.



,gvkey,companyid,startdate,enddate,entity_pname,ratingdate,ratingsymbol,ratingactionword,unsol
0,210835,18511.0,1901-01-01,2999-12-31,3i Group PLC,1995-12-07,AA-,New Rating,N
2,210835,18511.0,1901-01-01,2999-12-31,3i Group PLC,2001-12-20,A+,Downgrade,N
5,210835,18511.0,1901-01-01,2999-12-31,3i Group PLC,2008-11-11,A,Downgrade,N
6,210835,18511.0,1901-01-01,2999-12-31,3i Group PLC,2009-01-28,A-,Downgrade,N
7,210835,18511.0,1901-01-01,2999-12-31,3i Group PLC,2009-02-23,BBB+,Downgrade,N
...,...,...,...,...,...,...,...,...,...
90576,019045,41269602.0,1994-10-02,2005-09-30,Mitsubishi Chemical Corp.,1997-06-25,BBB,New Rating,N
90577,019045,41269602.0,1994-10-02,2005-09-30,Mitsubishi Chemical Corp.,2000-11-15,BBB-,Downgrade,N
90583,019045,41269602.0,1994-10-02,2005-09-30,Mitsubishi Rayon Co. Ltd.,2004-03-19,BBB,New Rating,N
90584,019045,41269602.0,1994-10-02,2005-09-30,Mitsubishi Rayon Co. Ltd.,2006-06-07,BBB+,Upgrade,N


In [49]:
## Check when GVKEY and CompanyID are not aligned one-to-one.
check = ratings3.groupby('gvkey')['companyid'].nunique()
selected_gvkeys = check[check > 1].index
duplicate_data = ratings3[ratings3['gvkey'].isin(selected_gvkeys)]
duplicate_data =duplicate_data.sort_values(['gvkey', 'companyid', 'ratingdate'], ascending = [True, True, True])

print(f'\nThe number of rows is {len(duplicate_data):,}.\n')
duplicate_data.head(25)
#duplicate_data


The number of rows is 922.



,gvkey,companyid,startdate,enddate,entity_pname,ratingdate,ratingsymbol,ratingactionword,unsol
84005,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,1996-03-13,BB+,New Rating,N
84007,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,1997-04-29,BBB-,Upgrade,N
84008,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,1997-11-12,BBB,Upgrade,N
84012,001081,527596.0,2000-04-19,2999-12-31,Abitibi-Consolidated Inc.,1997-11-17,BBB-,<NA>,N
84010,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,2000-04-20,BBB-,Downgrade,N
84015,001081,527596.0,2000-04-19,2999-12-31,Abitibi-Consolidated Inc.,2003-02-19,BB+,Downgrade,N
84011,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,2003-03-14,NR,Not Rated,N
84017,001081,527596.0,2000-04-19,2999-12-31,Abitibi-Consolidated Inc.,2004-03-04,BB,Downgrade,N
84018,001081,527596.0,2000-04-19,2999-12-31,Abitibi-Consolidated Inc.,2004-12-20,BB-,Downgrade,N
84019,001081,527596.0,2000-04-19,2999-12-31,Abitibi-Consolidated Inc.,2005-12-01,B+,Downgrade,N


Look closly to the data, focus on `gvkey = 001081`, we will find this is one single company. It changed `companyid` due to several times of aquisition. But the real problem is it has some ratingsymbol that shows `Not Rated`, which does not make much sense. I think we can drop these lines.

In [30]:
gvkey_target = '001081'
filtered_data = ratings3[ratings3['gvkey'] == gvkey_target].sort_values('ratingdate', ascending=True)
filtered_data


,gvkey,companyid,startdate,enddate,entity_pname,ratingdate,ratingsymbol,ratingactionword,unsol
90534,001081,932610.0,1997-05-31,2000-04-18,Stone Consolidated Corp.,1993-10-19,BB-,<NA>,N
90537,001081,932610.0,1997-05-31,2000-04-18,Stone Consolidated Corp.,1995-06-23,BB,<NA>,N
90539,001081,932610.0,1997-05-31,2000-04-18,Stone Consolidated Corp.,1995-11-01,BB+,<NA>,N
84005,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,1996-03-13,BB+,New Rating,N
84007,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,1997-04-29,BBB-,Upgrade,N
90541,001081,932610.0,1997-05-31,2000-04-18,Stone Consolidated Corp.,1997-06-10,NR,Not Rated,N
84008,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,1997-11-12,BBB,Upgrade,N
84012,001081,527596.0,2000-04-19,2999-12-31,Abitibi-Consolidated Inc.,1997-11-17,BBB-,<NA>,N
84010,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,2000-04-20,BBB-,Downgrade,N
84015,001081,527596.0,2000-04-19,2999-12-31,Abitibi-Consolidated Inc.,2003-02-19,BB+,Downgrade,N


In [34]:
df = ratings3.copy()

# Identify gvkeys associated with multiple companyids (indicative of mergers or acquisitions)
companyid_counts = df.groupby('gvkey')['companyid'].nunique()
changing_gvkeys = companyid_counts[companyid_counts > 1].index
df_filtered = df[df['gvkey'].isin(changing_gvkeys)].copy()

# Sort by gvkey and ratingdate to ensure correct chronological order
df_filtered = df_filtered.sort_values(['gvkey', 'ratingdate'])

df_filtered['is_nr'] = df_filtered['ratingsymbol'] == 'NR'
df_filtered['prev_is_rating'] = df_filtered.groupby('gvkey')['ratingsymbol'].shift(1).notna() & (df_filtered.groupby('gvkey')['ratingsymbol'].shift(1) != 'NR')
df_filtered['next_is_rating'] = df_filtered.groupby('gvkey')['ratingsymbol'].shift(-1).notna() & (df_filtered.groupby('gvkey')['ratingsymbol'].shift(-1) != 'NR')
to_remove = df_filtered['is_nr'] & df_filtered['prev_is_rating'] & df_filtered['next_is_rating']
indices_to_remove = df_filtered[to_remove].index

ratings3_cleaned = ratings3.drop(index=indices_to_remove)

gvkey_target = '001081'
filtered_data = ratings3_cleaned[ratings3['gvkey'] == gvkey_target].sort_values('ratingdate', ascending=True)
filtered_data


C:\Users\123\AppData\Local\Temp\ipykernel_28872\788672147.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = ratings3_cleaned[ratings3['gvkey'] == gvkey_target].sort_values('ratingdate', ascending=True)


,gvkey,companyid,startdate,enddate,entity_pname,ratingdate,ratingsymbol,ratingactionword,unsol
90534,001081,932610.0,1997-05-31,2000-04-18,Stone Consolidated Corp.,1993-10-19,BB-,<NA>,N
90537,001081,932610.0,1997-05-31,2000-04-18,Stone Consolidated Corp.,1995-06-23,BB,<NA>,N
90539,001081,932610.0,1997-05-31,2000-04-18,Stone Consolidated Corp.,1995-11-01,BB+,<NA>,N
84005,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,1996-03-13,BB+,New Rating,N
84007,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,1997-04-29,BBB-,Upgrade,N
84008,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,1997-11-12,BBB,Upgrade,N
84012,001081,527596.0,2000-04-19,2999-12-31,Abitibi-Consolidated Inc.,1997-11-17,BBB-,<NA>,N
84010,001081,527596.0,2000-04-19,2999-12-31,Donohue Inc.,2000-04-20,BBB-,Downgrade,N
84015,001081,527596.0,2000-04-19,2999-12-31,Abitibi-Consolidated Inc.,2003-02-19,BB+,Downgrade,N
84017,001081,527596.0,2000-04-19,2999-12-31,Abitibi-Consolidated Inc.,2004-03-04,BB,Downgrade,N


In [67]:
ratings3_sorted=ratings3_cleaned.sort_values(['gvkey', 'companyid', 'ratingdate'], ascending = [True, True, True])
print(f'\nThe number of rows is {len(ratings3_sorted):,}.\n')
ratings3_sorted


The number of rows is 46,140.



,gvkey,companyid,startdate,enddate,entity_pname,ratingdate,ratingsymbol,ratingactionword,unsol
12766,001004,168154.0,1901-01-01,2999-12-31,AAR Corp.,1995-01-23,BBB-,Downgrade,N
12768,001004,168154.0,1901-01-01,2999-12-31,AAR Corp.,1997-07-03,BBB,Upgrade,N
12770,001004,168154.0,1901-01-01,2999-12-31,AAR Corp.,2002-01-10,BBB-,Downgrade,N
12772,001004,168154.0,1901-01-01,2999-12-31,AAR Corp.,2003-04-18,BB-,Downgrade,N
12776,001004,168154.0,1901-01-01,2999-12-31,AAR Corp.,2006-10-13,BB,Upgrade,N
...,...,...,...,...,...,...,...,...,...
80220,358690,1798826927.0,1901-01-01,2999-12-31,RENK Group AG,2024-11-27,BB,Upgrade,Y
80137,358790,707536328.0,1901-01-01,2999-12-31,Mandatum Plc,2024-03-21,BBB+,New Rating,Y
74028,359443,31089174.0,1901-01-01,2999-12-31,Indian Renewable Energy Development Agency Ltd.,2024-08-29,BBB-,New Rating,Y
80228,359645,1844488027.0,1901-01-01,2999-12-31,Syensqo,2023-12-11,BBB+,New Rating,Y


Ensuring that the ratingdate falls within the valid date range (between startdate and enddate), ensures that we are only considering ratings that are actively valid during a specific time period.

In [71]:
import datetime

def to_python_date(x):
    # If x is already a Python date, just return it
    if isinstance(x, datetime.date):
        return x
    
    # If x is missing (NaN, None, etc.), return None
    if pd.isnull(x):
        return None
    
    # Convert x to string in case it's not
    s = str(x)
    
    # Here we assume the format is 'YYYY-MM-DD'
    # so split by '-' and convert to int
    try:
        year, month, day = map(int, s.split('-'))
        return datetime.date(year, month, day)
    except:
        # If for some reason it doesn't parse, return None
        return None

# 1) Convert ratingdate, startdate, enddate into Python date objects
ratings3['ratingdate'] = ratings3['ratingdate'].apply(to_python_date)
ratings3['startdate']  = ratings3['startdate'].apply(to_python_date)
ratings3['enddate']    = ratings3['enddate'].apply(to_python_date)

# 2) Now sort by gvkey, ratingdate
ratings3_sorted = ratings3.sort_values(['gvkey', 'ratingdate'])

# 3) Filter rows where ratingdate is between startdate and enddate (inclusive)
#    This comparison works fine with Python date objects, even with year 2999.
ratings3_sorted = ratings3_sorted[
    (ratings3_sorted['ratingdate'] >= ratings3_sorted['startdate']) &
    (ratings3_sorted['ratingdate'] <= ratings3_sorted['enddate'])
]

ratings4 = ratings3_sorted
print("The number of rows is", len(ratings4))
ratings4.head(10)


The number of rows is 639


C:\Users\123\AppData\Local\Temp\ipykernel_28872\3714524624.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings3['ratingdate'] = ratings3['ratingdate'].apply(to_python_date)
C:\Users\123\AppData\Local\Temp\ipykernel_28872\3714524624.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings3['startdate']  = ratings3['startdate'].apply(to_python_date)
C:\Users\123\AppData\Local\Temp\ipykernel_28872\3714524624.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,gvkey,companyid,startdate,enddate,entity_pname,ratingdate,ratingsymbol,ratingactionword,unsol
90541,001081,932610.0,1997-05-31,2000-04-18,Stone Consolidated Corp.,1997-06-10,NR,Not Rated,N
90127,001177,26589755.0,1901-01-01,2000-12-13,Aetna Inc. (Old),1997-07-25,A-,New Rating,N
90128,001177,26589755.0,1901-01-01,2000-12-13,Aetna Inc. (Old),1997-08-14,A,<NA>,N
89489,001995,3117990.0,1901-01-01,1999-05-02,Baltimore Gas and Electric Co.,1990-04-16,A+,Downgrade,N
87733,002337,171445.0,1901-01-01,2007-10-28,Bowater Inc.,1991-12-09,BBB+,Downgrade,N
87735,002337,171445.0,1901-01-01,2007-10-28,Bowater Inc.,1992-08-04,BBB,Downgrade,N
87737,002337,171445.0,1901-01-01,2007-10-28,Bowater Inc.,1993-11-04,BBB-,Downgrade,N
87739,002337,171445.0,1901-01-01,2007-10-28,Bowater Inc.,1995-10-11,BBB,Upgrade,N
87744,002337,171445.0,1901-01-01,2007-10-28,Bowater Inc.,2002-09-23,BBB-,Downgrade,N
87745,002337,171445.0,1901-01-01,2007-10-28,Bowater Inc.,2003-02-19,BB+,Downgrade,N


In [66]:
# Convert date columns to datetime using .loc to ensure the original DataFrame is modified
ratings3.loc[:, 'ratingdate'] = pd.to_datetime(ratings3['ratingdate'], errors='coerce')  # Convert ratingdate column
ratings3.loc[:, 'startdate'] = pd.to_datetime(ratings3['startdate'], errors='coerce')    # Convert startdate column
ratings3.loc[:, 'enddate'] = pd.to_datetime(ratings3['enddate'], errors='coerce')        # Convert enddate column

# Re-check the data types of the three date columns
print("Data type of 'ratingdate':", ratings3['ratingdate'].dtype)  # Expected: datetime64[ns]
print("Data type of 'startdate':", ratings3['startdate'].dtype)    # Expected: datetime64[ns]
print("Data type of 'enddate':", ratings3['enddate'].dtype)        # Expected: datetime64[ns]


Data type of 'ratingdate': object
Data type of 'startdate': object
Data type of 'enddate': object


In [10]:
ratings4['ratingdate'] = pd.to_datetime(ratings4['ratingdate'])
ratings4['ratingdate'] = ratings4['ratingdate'].dt.strftime('%Y-%m-%d')
ratings4_with_primarykeys = ratings4.set_index(['gvkey', 'ratingdate'])
ratings5 = ratings4_with_primarykeys

print(f'\nThe number of rows is {len(ratings5):,}.\n')
ratings5.head(20)


The number of rows is 44,661.



companyid   startdate     enddate  \
gvkey  ratingdate                                      
001004 1995-01-23   168154.0  1901-01-01  2999-12-31   
       1997-07-03   168154.0  1901-01-01  2999-12-31   
       2002-01-10   168154.0  1901-01-01  2999-12-31   
       2003-04-18   168154.0  1901-01-01  2999-12-31   
       2006-10-13   168154.0  1901-01-01  2999-12-31   
       2015-04-03   168154.0  1901-01-01  2999-12-31   
       2020-05-28   168154.0  1901-01-01  2999-12-31   
       2023-11-16   168154.0  1901-01-01  2999-12-31   
       2024-02-14   168154.0  1901-01-01  2999-12-31   
001034 2007-03-23   246886.0  1901-01-01  2999-12-31   
       2009-02-26   246886.0  1901-01-01  2999-12-31   
001036 2000-01-19   310609.0  1901-01-01  2999-12-31   
       2001-05-25   310609.0  1901-01-01  2999-12-31   
001045 1990-08-20   168569.0  1901-01-01  2999-12-31   
       1991-04-05   168569.0  1901-01-01  2999-12-31   
       1992-06-01   168569.0  1901-01-01  2999-12-31   
       1993-03-11   168569.0  1901-01-01  2999-12-31   
       1997-06-16   168569.0  1901-01-01  2999-12-31   
       2001-09-20   168569.0  1901-01-01  2999-12-31   
       2002-06-28   168569.0  1901-01-01  2999-12-31   

                                      entity_pname ratingsymbol  \
gvkey  ratingdate                                                 
001004 1995-01-23                        AAR Corp.         BBB-   
       1997-07-03                        AAR Corp.          BBB   
       2002-01-10                        AAR Corp.         BBB-   
       2003-04-18                        AAR Corp.          BB-   
       2006-10-13                        AAR Corp.           BB   
       2015-04-03                        AAR Corp.          BB+   
       2020-05-28                        AAR Corp.           BB   
       2023-11-16                        AAR Corp.          BB+   
       2024-02-14                        AAR Corp.           BB   
001034 2007-03-23                    Alpharma Inc.           B+   
       2009-02-26                    Alpharma Inc.           NR   
001036 2000-01-19  United Dominion Industries Inc.          BBB   
       2001-05-25  United Dominion Industries Inc.           NR   
001045 1990-08-20     American Airlines Group Inc.           A-   
       1991-04-05     American Airlines Group Inc.         BBB+   
       1992-06-01     American Airlines Group Inc.          BBB   
       1993-03-11     American Airlines Group Inc.          BB+   
       1997-06-16     American Airlines Group Inc.         BBB-   
       2001-09-20     American Airlines Group Inc.           BB   
       2002-06-28     American Airlines Group Inc.          BB-   

                  ratingactionword unsol  
gvkey  ratingdate                         
001004 1995-01-23        Downgrade     N  
       1997-07-03          Upgrade     N  
       2002-01-10        Downgrade     N  
       2003-04-18        Downgrade     N  
       2006-10-13          Upgrade     N  
       2015-04-03          Upgrade     N  
       2020-05-28        Downgrade     N  
       2023-11-16          Upgrade     N  
       2024-02-14        Downgrade     Y  
001034 2007-03-23       New Rating     N  
       2009-02-26        Not Rated     N  
001036 2000-01-19       New Rating     N  
       2001-05-25        Not Rated     N  
001045 1990-08-20        Downgrade     N  
       1991-04-05        Downgrade     N  
       1992-06-01        Downgrade     N  
       1993-03-11             None     N  
       1997-06-16             None     N  
       2001-09-20        Downgrade     N  
       2002-06-28        Downgrade     N

In [11]:
# Example
specific_gvkey = '001004'  
specific_ratingdate = '1995-01-23' 
specific_row = ratings5.loc[(specific_gvkey, specific_ratingdate)].to_frame().T

specific_row

,,companyid,startdate,enddate,entity_pname,ratingsymbol,ratingactionword,unsol
001004,1995-01-23,168154.0,1901-01-01,2999-12-31,AAR Corp.,BBB-,Downgrade,N


Additional Ratings:
- NR (Not Rated): The issuer has not been rated by S&P. It can be considered a withdrawal.
- D indicates a full default, where the issuer has completely failed to meet its obligations.
- SD (Selective Default): The issuer has selectively defaulted on one or more obligations but remains current on others. This is considered a partial default.
- R (Regulatory Review): The rating is under review by the regulatory authorities, often due to some uncertainty surrounding the issuer’s financial position.

P.S. We'll consider 'R', which represents regulatory supervision, as a default indicator, as this is important for banks and insurance companies, which are not allowed to use bankruptcy laws in the same way as other corporations.


In [12]:
## test 1:
defaults1 = ratings5[ratings5.ratingsymbol.isin(['D', 'SD', 'R'])]
defaults1['default_flag'] = 1

print(f'\nThe number of rows is {len(defaults1):,}.\n')
defaults1


The number of rows is 1,677.



/var/folders/2h/ldznmrln3pbbxyk7clkx_wth0000gn/T/ipykernel_68334/1815464562.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defaults1['default_flag'] = 1


,,companyid,startdate,enddate,entity_pname,ratingsymbol,ratingactionword,unsol,default_flag
gvkey,ratingdate,,,,,,,,
001045,2011-11-29,168569.0,1901-01-01,2999-12-31,American Airlines Group Inc.,D,Downgrade,N,1
001059,1999-01-27,58171780.0,1901-01-01,2999-12-31,"A.P.S., Inc.",D,New Rating,N,1
001081,2009-03-31,527596.0,2000-04-19,2999-12-31,Abitibi-Consolidated Inc.,D,Downgrade,N,1
001308,2009-06-29,1066957.0,2001-05-10,2999-12-31,Allis-Chalmers Energy Inc.,SD,Downgrade,N,1
001382,2002-06-25,169142.0,1994-08-26,2999-12-31,US Airways Group Inc.,SD,Downgrade,N,1
...,...,...,...,...,...,...,...,...,...
323554,2022-11-29,39575377.0,1901-01-01,2999-12-31,CORESTATE Capital Holding S.A.,D,Downgrade,N,1
324989,2018-05-17,268074105.0,1901-01-01,2999-12-31,GetBack S.A.,D,New Rating,N,1
325248,2024-03-27,372433823.0,1901-01-01,2999-12-31,Samhallsbyggnadsbolaget i Norden AB (publ),SD,Downgrade,N,1


In [13]:
## test 2:
from dateutil.relativedelta import relativedelta

# selects ratings that were active on a specific date
date = dt.date(2023, 12, 31) 
date_plus_one = date + relativedelta(years = 1)

print(date)
print(date_plus_one)

# print(format(date))
print(type(date))


2023-12-31
2024-12-31
<class 'datetime.date'>


In [14]:
date = date.strftime('%Y-%m-%d')
date_plus_one = date_plus_one.strftime('%Y-%m-%d')

print(date)
print(date_plus_one)
#print(format(date))
print(type(date))

2023-12-31
2024-12-31
<class 'str'>


In [26]:
defaults2 = ratings5[ratings5.ratingsymbol.isin(['D', 'SD', 'R'])]
defaults2 = defaults2[(defaults2.index.get_level_values('ratingdate') > date) & 
                      (defaults2.index.get_level_values('ratingdate') <= date_plus_one)]
defaults2['default_flag'] = 1

print(f'\nThe number of rows is {len(defaults2):,}.\n')
defaults2.head()


The number of rows is 52.



companyid   startdate     enddate             entity_pname  \
gvkey  ratingdate                                                               
001920 2024-08-13   253132.0  1901-01-01  2999-12-31       Avon Products Inc.   
002884 2024-03-22   260149.0  1901-01-01  2999-12-31  Lumen Technologies Inc.   
       2024-10-02   260149.0  1901-01-01  2999-12-31  Lumen Technologies Inc.   
004523 2024-03-19   270700.0  1901-01-01  2999-12-31        Jo-Ann Stores LLC   
012669 2024-01-31    26145.0  1901-01-01  2999-12-31     Carmike Cinemas Inc.   

                  ratingsymbol ratingactionword unsol  default_flag  
gvkey  ratingdate                                                    
001920 2024-08-13            D        Downgrade     N             1  
002884 2024-03-22           SD        Downgrade     N             1  
       2024-10-02           SD        Downgrade     N             1  
004523 2024-03-19            D        Downgrade     N             1  
012669 2024-01-31           SD        Downgrade     N             1

In [21]:
# this is where we pull down the financials for use in modeling
# we should add anything additional that we can think of

sql_financials = """
SELECT 
    gvkey, 
    datadate,          -- Statement Date (Fiscal Year-End)
    fyear,             -- Fiscal Year
    fyr,               -- Fiscal Year-End Month
    at,                -- Total Assets
    lt,                -- Total Liabilities
    ceq,               -- Common Equity
    act,               -- Current Assets
    lct,               -- Current Liabilities
    invt,              -- Inventories
    rect,              -- Accounts Receivable
    ap,                -- Accounts Payable
    dlc,               -- Debt in Current Liabilities (Short-Term Debt)
    dltt,              -- Debt in Long-Term Liabilities
    dltis,             -- Debt Issued
    dvt,               -- Debt Valuation
    che,               -- Cash and Short-Term Investments
    xint,              -- Interest Expense
    xrd,               -- Research and Development Expense
    xsga,              -- Selling, General, and Administrative Expenses
    oibdp,             -- Operating Income Before Depreciation
    ebit,              -- Earnings Before Interest and Taxes
    sale,              -- Sales/Revenue
    cogs,              -- Cost of Goods Sold
    ni,                -- Net Income
    oancf,             -- Operating Cash Flow
    fincf,             -- Financing Cash Flow
    csho,              -- Common Shares Outstanding
    prcc_f,            -- Price Close
    'Annual' AS freq   -- Data Frequency Indicator
FROM 
    comp.funda
WHERE  
    indfmt = 'INDL'     -- Industrial Format
    AND datafmt = 'STD' -- Standardized Format
    AND popsrc = 'D'     -- Domestic Companies
    AND consol = 'C'     -- Consolidated Data

    AND fyear >= 1990    -- Adjust the starting fiscal year as needed
"""

financials = conn.raw_sql(sql_financials)

print(f'\nThe number of rows is {len(financials):,}.\n')
financials.head(10)


The number of rows is 396,713.



,gvkey,datadate,fyear,fyr,at,lt,ceq,act,lct,invt,...,oibdp,ebit,sale,cogs,ni,oancf,fincf,csho,prcc_f,freq
0,001004,1991-05-31,1990,5,379.958,186.180,193.778,268.399,79.227,156.133,...,41.957,33.701,466.542,366.040,14.801,36.891,-32.043,15.891,14.124999,Annual
1,001009,1990-10-31,1990,10,32.335,26.073,6.262,10.047,8.382,2.355,...,6.254,4.283,40.522,30.633,2.135,2.075,3.400,2.526,5.999998,Annual
2,001010,1990-12-31,1990,12,1728.888,1516.287,210.399,523.502,421.777,36.860,...,177.870,121.123,312.309,106.729,-75.287,28.377,83.508,0.015,NaN,Annual
3,001011,1990-12-31,1990,12,7.784,7.117,0.667,1.247,1.246,0.663,...,-0.570,-1.270,3.635,1.125,-1.869,-0.781,2.001,3.556,1.188000,Annual
4,001013,1990-10-31,1990,10,181.665,47.652,134.013,102.525,37.335,33.845,...,49.725,37.745,259.802,121.822,22.903,34.695,-0.312,13.289,16.375000,Annual
5,001014,1990-06-30,1990,6,13.491,6.076,7.415,5.025,2.739,0.000,...,1.475,0.701,2.644,1.169,2.037,0.677,-0.123,0.685,11.000000,Annual
6,001017,1991-02-28,1990,2,118.120,73.727,44.393,55.040,32.615,4.292,...,12.892,6.621,144.258,107.343,2.574,9.994,-3.112,3.882,5.125000,Annual
7,001019,1990-12-31,1990,12,21.716,13.293,8.423,9.075,3.001,0.284,...,4.649,2.692,22.667,12.090,1.721,4.083,-2.219,0.206,NaN,Annual
8,001021,1990-06-30,1990,6,34.591,25.545,7.751,24.684,16.208,15.463,...,4.470,3.145,48.292,31.214,0.422,0.865,-0.655,4.755,1.561999,Annual
9,001025,1991-02-28,1990,2,5.631,2.832,2.799,5.323,2.704,2.798,...,0.614,0.573,11.285,7.111,0.126,0.549,-0.289,4.207,NaN,Annual


In [22]:
# this is where we get information on industry and region

sql_info = """
SELECT 
    gvkey,
    conm,       -- Company Name
    fic,        -- Current ISO Country Code - Incorporation
    fyrc,       -- month of fiscal year end
    gsector,    -- GIC Sector  (Global Identification Code)
    ggroup,     -- GIC Group
    gind,       -- GIC Industries
    idbflag,    -- International, Domestic, Both Indicator
    incorp,     -- Current State/Province of Incorporation Code
    loc,        -- Current ISO Country Code - Headquarters
    naics,      -- North American Industry Classification Code
    sic,        -- Standard Identification Code
    state       -- State/Province
FROM 
    comp.company
"""
# Note that GICs is the current industry leader while SIC and NAICs are considered outdated

info = conn.raw_sql(sql_info)

print(f'\nThe number of rows is {len(info):,}.\n')
info.head()


The number of rows is 54,885.



,gvkey,conm,fic,fyrc,gsector,ggroup,gind,idbflag,incorp,loc,naics,sic,state
0,001000,A & E PLASTIK PAK INC,USA,12,None,None,None,D,None,USA,None,3089,None
1,001001,A & M FOOD SERVICES INC,USA,12,25,2530,253010,D,None,USA,722,5812,OK
2,001002,AAI CORP,USA,12,None,None,None,D,None,USA,None,3825,MD
3,001003,A.A. IMPORTING CO INC,USA,1,25,2550,255040,D,DE,USA,442110,5712,MO
4,001004,AAR CORP,USA,5,20,2010,201010,D,DE,USA,423860,5080,IL
